# RNN

## 程式參考來源：
- https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
- https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN
- https://pytorch.org/text/stable/vocab.html
- https://pytorch.org/text/stable/functional.html#to-tensor
- https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np

C:\Users\a4022\anaconda3\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Embedding test

In [3]:
# 內含值(0~5): 為在詞彙中的索引值
## nn.Embedding第一個參數為6，表示詞彙表含6個單字
## nn.Embedding第二個參數為5，表示每個單字以5個實數表示
x = torch.LongTensor([[0,1,2], [3,4,5]])
print(x.shape)
embeds = nn.Embedding(6, 5) 
print(embeds(x))
print(embeds(x).shape)

torch.Size([2, 3])
tensor([[[ 0.5907, -0.8356,  1.3009,  0.0583, -0.6387],
         [ 1.1769,  0.6917, -0.9654,  0.5125, -1.0630],
         [-1.8748,  0.3328, -0.9192,  0.5780,  0.6709]],

        [[ 0.2040, -1.3705, -2.3143, -0.8078, -0.0714],
         [-1.1596, -0.4426, -0.1378,  0.2899,  0.0822],
         [-1.4597,  0.1403, -1.1047, -0.1358,  1.0442]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])


In [5]:
# 隨機亂數，預設為標準常態分配 N(0,1)
embeds.weight

Parameter containing:
tensor([[ 0.5907, -0.8356,  1.3009,  0.0583, -0.6387],
        [ 1.1769,  0.6917, -0.9654,  0.5125, -1.0630],
        [-1.8748,  0.3328, -0.9192,  0.5780,  0.6709],
        [ 0.2040, -1.3705, -2.3143, -0.8078, -0.0714],
        [-1.1596, -0.4426, -0.1378,  0.2899,  0.0822],
        [-1.4597,  0.1403, -1.1047, -0.1358,  1.0442]], requires_grad=True)

In [8]:
x = torch.LongTensor([[1,2,3], [4,5,6]])
embeds = nn.Embedding(7, 5) 
print(embeds(x))

tensor([[[ 0.0444,  1.8696, -0.2251, -1.5209,  0.7612],
         [ 0.9068,  0.0946,  0.5470,  0.7086,  1.3345],
         [-2.5969, -0.3708, -0.6219, -0.3268, -0.1398]],

        [[ 1.3420, -0.7199,  0.5901,  0.5138,  0.9663],
         [ 1.0496, -0.2049,  0.2492,  0.6211,  0.7383],
         [ 0.3613,  0.0763, -0.2415, -1.7523, -0.7044]]],
       grad_fn=<EmbeddingBackward0>)


In [9]:
embeds = nn.Embedding(6, 5) 
x1 = torch.LongTensor([[0,1,2]])
x2 = torch.LongTensor([[3,4]])
print(embeds(x1))
print(embeds(x2))
embeds.weight

tensor([[[ 0.3233, -0.1667, -2.0280,  1.0196,  0.6468],
         [ 0.4878, -1.4289,  0.5792, -0.4249,  1.8793],
         [-0.3445, -0.9100,  1.0745, -1.1607, -0.0575]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-1.2572, -0.3359, -2.4758, -0.8704,  1.4450],
         [ 0.9431,  0.2468, -1.8133,  1.6846, -0.8438]]],
       grad_fn=<EmbeddingBackward0>)


Parameter containing:
tensor([[ 0.3233, -0.1667, -2.0280,  1.0196,  0.6468],
        [ 0.4878, -1.4289,  0.5792, -0.4249,  1.8793],
        [-0.3445, -0.9100,  1.0745, -1.1607, -0.0575],
        [-1.2572, -0.3359, -2.4758, -0.8704,  1.4450],
        [ 0.9431,  0.2468, -1.8133,  1.6846, -0.8438],
        [ 1.3610, -1.0247,  0.7604, -1.1029, -0.8650]], requires_grad=True)

In [11]:
embeds = nn.Embedding(6, 5, 5) 
x1 = torch.LongTensor([[0,1,2]])
x2 = torch.LongTensor([[3,4]])
x3 = torch.LongTensor([[3,4]])
print(embeds(x1))
print(embeds(x1).shape)
print(embeds(x2))
print(embeds(x2).shape)
print(embeds(x3))
print(embeds(x3).shape)
embeds.weight

tensor([[[ 0.5837,  0.6748,  0.0295,  0.6885,  0.9582],
         [ 1.7321,  0.0511,  0.8235, -0.9037,  2.3132],
         [ 1.2274,  1.0231,  1.1000,  0.6160, -0.6655]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 3, 5])
tensor([[[ 0.1158, -1.0572,  0.9541,  0.5428,  0.3446],
         [-0.2312,  0.6069, -0.0030,  0.8377, -1.4637]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 2, 5])
tensor([[[ 0.1158, -1.0572,  0.9541,  0.5428,  0.3446],
         [-0.2312,  0.6069, -0.0030,  0.8377, -1.4637]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 2, 5])


Parameter containing:
tensor([[ 0.5837,  0.6748,  0.0295,  0.6885,  0.9582],
        [ 1.7321,  0.0511,  0.8235, -0.9037,  2.3132],
        [ 1.2274,  1.0231,  1.1000,  0.6160, -0.6655],
        [ 0.1158, -1.0572,  0.9541,  0.5428,  0.3446],
        [-0.2312,  0.6069, -0.0030,  0.8377, -1.4637],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], requires_grad=True)

In [14]:
# 測試資料
word_to_ix = {"hello": 0, "world": 1}
# 詞彙表(vocabulary)含2個單字, 轉換為5維的向量
embeds = nn.Embedding(2, 5) 
# 測試 hello
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(lookup_tensor)
print(hello_embed)
print(embeds.weight)

tensor([[-1.0467,  2.7163,  1.1429, -0.5163,  1.8922]],
       grad_fn=<EmbeddingBackward0>)
Parameter containing:
tensor([[-1.0467,  2.7163,  1.1429, -0.5163,  1.8922],
        [-1.5272,  0.1162,  0.4437,  1.9996,  0.6836]], requires_grad=True)
